`Autor: Piotr Koneczny (student PŁ, Wydział FTIMS, Kierunek: Matematyka Stosowana, studia II stopnia magisterskie stacjonarne dzienne, nr albumu: 258086)`

$$\text{\huge{Algorytm Simplex}}$$

Bibliografia:

[1] Paweł Zieliński, Metody Optymalizacji, wykłady nr 4 i 5. „ZPR PWr – Zintegrowany Program Rozwoju Politechniki Wrocławskiej ", 2024. https://cs.pwr.edu.pl/zielinski/lectures/om/wyklady4_5.pdf

[2] Wojda, A. P. (2013). Wykłady z programowania liniowego. Wydział Matematyki Stosowanej AGH. https://wms.mat.agh.edu.pl/˜wojda/Pl3.pdf

[3] Przemysław Gordinowicz, Materiały pomocnicze do wykładu nr 1 (LP, MIP) w ramach przedmiotu "Algorytmy optymalizacji dyskretnej", prowadzonego dla studentów II semestru na studiach II stopnia magisterskich dziennych, na kierunku "Matematyka stosowana". Politechnika Łódzka, 2025.

[4] Arthur David Snider, Basics of Optimization Theory, Springer Cham, 2023, DOI: 10.1007/978-3-031-29219-4. Chapter 2

[5] Dennis Amematekpor, The Simplex Algorithm | Linear Programming. https://youtu.be/fTKuIXOHbwM

---

In [ ]:
#Wczytujemy potrzebne biblioteki

import numpy as np                      #główna biblioteka do obliczeń
from copy import deepcopy               #metoda pomocnicza do tworzenia głębokich kopii obiektów w razie potrzeby
from scipy.optimize import linprog      #solver LP z scipy do porównania wyników
from itertools import combinations      #do "brutalnego" przeszukiwania

---

$\text{\large{Krótkie przypomnienie i uporządkowanie podstawowych wiadomości z zakresu programowania liniowego. Podstawowe oznaczenia i ustalenia}}$

Ogólny problem `programowania matematycznego` (a więc rozwiązywania zadań optymalizacyjnych z określoną funkcją celu) można sformułować następująco, jak w pracy [2]:

$\textit{Niech } X \textit{ będzie dowolnym zbiorem i niech }f \textit{ będzie funkcją zdefiniowaną w zbiorze } X
\textit{o wartościach w zbiorze liczb rzeczywistych.} \\ \textit{Znajdź wartość maksymalną } f(x) \textit{ dla } x \in X.$

Problemy optymalizacyjne mogą być formułowane na różne sposoby, w zależności od przyjętych założeń. 

Przykładowo, maksymalizacja funkcji $f \colon X \to \mathbb{R}$ jest równoważna minimalizacji funkcji $g \colon X \to \mathbb{R}$, zdefiniowanej jako $g(x) = -f(x)$ dla każdego $x \in X$.

Jednym z najważniejszych i najlepiej poznanych przypadków programowania matematycznego jest programowanie liniowe (ang. $linear \, programming$, w skrócie $LP$). 

`W problemach LP zakłada się, że:`

1. przestrzeń dopuszczalna $X$ jest podzbiorem przestrzeni euklidesowej $\mathbb{R}^n$,

2. funkcja celu $f$ jest funkcją liniową.

`W takich przypadkach zbiór rozwiązań dopuszczalnych często przyjmuje postać wielościanu` — zbioru wyznaczonego przez układ skończonej liczby nierówności liniowych. Warto zauważyć, że rozwiązania optymalne w problemach $LP$ bardzo często znajdują się w wierzchołkach tego wielościanu, co ma kluczowe znaczenie dla działania algorytmu simpleks.

---

Przypomnimy teraz formalną definicję problemu programowania liniowego, która pozwoli nam dokładnie zrozumieć strukturę tego typu zadań i uzasadnić zastosowanie metody simpleks. Ustalimy przy okazji i uporządkujemy pewne oznaczenia, które będą się pojawiały w kontekście problemów $LP$. 

Wklejam ją z preliminariów mojej (właśnie powstającej) pracy magisterskiej (o zastosowaniu $MIP$ - Mixed Integer Programming - do harmonogramowania zajęć uczelni - zapewne poniższą implementację algorytmu Simplex wykorzystam później w konstrukcji programu układającego plan zajęć, który buduję w ramach mojej pracy i w ramach realizacji Własnego Funduszu Stypendialnego, przyznanego mi na rok akademicki 2024/2025).

![image.png](projekt_alg_Simplex_zdj/definicja_LP_z_mojej_mgr.png)

Algorytm Simplex, który omówimy za chwilę, rozwiązuje problemy $LP$ zapisane w postaci, gdzie wszystkie ograniczenia mają postać równości: $A\mathbf{x} = \mathbf{b}$, a nie nierówności $A\mathbf{x} \le \mathbf{b}$.

Jeżeli mamy problem $LP$ zadany w takiej postaci:

$$\begin{cases} \text{zmaksymalizować:} \quad c^T \bf{x} \\ \text{przy warunkach:} \quad \, \, \, \, A \cdot \bf{x} = \bf{b} \\ \qquad \qquad \qquad \, \quad \quad \, \, \, \, \quad \, \, \, \, \bf{x} \ge \theta_{\mathbb{R}^{n}} \end{cases}$$

to mówimy, że ten problem $LP$ jest zadany w `postaci kanonicznej`.

`W dalszej części tekstu będziemy zamiennie używać określeń: `

- `postać standardowa/nierównościowa, odwołując się do postaci (1.3.2) na screenie powyżej,`

- `postać kanoniczna/równościowa, odwołując się do postaci przedstawionej bezpośrednio powyżej, ale już pod screenem.`

`Problem $LP$ w postaci standardowej (przy powyższym nazewnictwie i oznaczeniach) można rozszerzyć do postaci kanonicznej (równościowej), zwiększając jego wymiar.`

Konkretnie mówiąc, robi się to dzięki wprowadzeniu tzw. `zmiennych uzupełniających/zmiennych zapasu/zmiennych luzu/slacków` (od ang. $\textit{slack variables}$).

Zamiana nierówności na równości poprzez dodanie zmiennych slack umożliwia reprezentację przestrzeni rozwiązań jako zbioru przecięć hiperpłaszczyzn, co jest kluczowe dla działania algorytmu Simplex, który porusza się od wierzchołka do wierzchołka tego zbioru.

Ponieważ mamy $m$ ograniczeń składających się na nierówność $A\bf{x} \le \bf{b}$ (ograniczeń jest tyle samo, ile współrzędnych ma kolumnowy wektor $\bf{b}$), wprowadźmy zmienne uzupełniające $s_1, \ldots, s_m$. Przyjmijmy również oznaczenie $\bf{s} := (s_1, \ldots, s_m)^T \in \R^m$. 

Oznaczmy również $\hat{\bf{c}} := (c_1, \ldots, c_n, c_{n+1}, \ldots, c_{n+m})^T \in \mathbb{R}^{n+m}$, a także $\hat{\bf{x}} := [\bf{x} \quad \bf{s}]^T$ oraz $\hat{A} := [A \mid I_m]$. Przy tych oznaczeniach, tak wygląda sformułowanie problemu (1.3.2) w postaci kanonicznej:

$$\begin{cases} \text{zmaksymalizować:} \quad \hat{c}^T \hat{\bf{x}} \\ \text{przy warunkach:} \quad \, \, \, \, \hat{A} \cdot \hat{\bf{x}} = \bf{b} \\ \qquad \qquad \qquad \, \quad \quad \, \, \, \, \quad \, \, \, \, \bf{x} \ge \theta_{\mathbb{R}^{n+m}} \end{cases}$$

Czyli w istocie rzeczy, rozszerzenie wyjściowego problemu $LP$ (w postaci standardowej, tzn. powyższej 1.3.2) do postaci kanonicznej dokonuje się poprzez zamianę nierówności $A\bf{x} \le \bf{b}$ na równość $\hat{A} \cdot \hat{\bf{x}} = \bf{b}$. 

Taką postać tego problemu $LP$ (jak w 1.3.2) nazywamy właśnie jego $\emph{postacią kanoniczną}$.

Poniżej definiujemy funkcję pomocniczą `preprocess_general_lp`, zamieniającą warunki mieszanych typów ("$\le$", "$\ge$", "$=$") na jedną wspólną nierówność w postaci macierzowej: $A\bf{x} \le \bf{b}$.

`Poniżej własnoręcznie zaimplementowany algorytm Simplex` najpierw przekształca wyjściowe ograniczenia do jednego wspólnego $A\bf{x} \le \bf{b}$ za pomocą powyżej wspomnianej funkcji. 

Następnie, w celu znalezienia rozwiązania takiego problemu $LP$ (w postaci standardowej), "zamienia" go na (równoważny) problem $LP$ w postaci kanonicznej. 

W dalszej kolejności rozwiązuje ten problem w postaci kanonicznej (bądź wskazuje, że nie istnieje jego optymalne rozwiązanie), automatycznie uzyskując też rozwiązanie wyjściowego problemu $LP$ (w postaci standardowej) i `zwraca rozwiązanie oryginalnego problemu (w postaci standardowej), odrzucając współczynniki dotyczące zmiennych uzupełniających`.

Przy powyższych oznaczeniach i ustaleniach, zbiór $\{ \bf{x} \in \mathbb{R}^n \colon A\bf{x} \le \bf{b}, \, \bf{x} \ge \theta_{\mathbb{R}^n} \}$ nazywamy $\emph{zbiorem rozwiązań dopuszczalnych}$ problemu $LP$, jak w (1.3.2), a elementy tego zbioru nazywamy $\emph{rozwiązaniami dopuszczalnymi}$ takiego problemu $LP$. 

Zbiór rozwiązań dopuszczalnych problemu $LP$ w postaci kanonicznej oczywiście definiuje się analogicznie, zastępując nierówność $\le$ równością.

Rozwiązanie dopuszczalne problemu $LP$ nazwiemy jego $\emph{optymalnym rozwiązaniem(dopuszczalnym)}$, jeżeli funkcja celu przyjmuje dla niego największą wartość w zbiorze dopuszczalnym.

---

$\text{\large{Teoria dotycząca algorytmu Simplex}}$

Zacznijmy może od tego, że `sympleks to uogólnienie odcinka, trójkąta i czworościanu na dowolne wymiary`.

Intuicyjnie, $k$-wymiarowym sympleksem nazywamy $k$-wymiarowy wielościan, który jest wypukłą otoczką swoich $k+1$ wierzchołków.

![image.png](projekt_alg_Simplex_zdj/sympleks_definicja.png)

Rys. 1. Przykład 3-wymiarowego sympleksu (źródło: https://upload.wikimedia.org/wikipedia/commons/2/25/Tetrahedron.png).

`Algorytm sympleks, który w ramach tej pracy omawiamy, będziemy też nazywać wymiennie: "algorytmem Simplex", od jego angielskiej nazwy.`

Jak już wspomnieliśmy chwilę wcześniej, algorytm sympleks rozwiązuje problem $LP$ w postaci kanonicznej (bądź stwierdza, że nie istnieją żadne rozwiązania problemu). Jak słusznie zauważono w [1]:

$\textit{Idea algorytmu sympleks polega na inteligentnym przejrzeniu wierzchołków zbioru rozwiązań dopuszczalnych i wybraniu wierzchołka,} \\ \textit{w którym wartość funkcji celu jest optymalna.}$

Każdy wierzchołek zbioru rozwiązań dopuszczalnych odpowiada pewnemu rozwiązaniu - takie rozwiązanie nazywamy `bazowym rozwiązaniem dopuszczalnym`. 

Bazowe rozwiązanie dopuszczalne $LP$ nazwiemy $\emph{bazowym rozwiązaniem optymalnym}$, jeżeli jest ono (dopuszczalnym) rozwiązaniem optymalnym tego $LP$.

Dla porządku, przypominamy z wykładu [3] twierdzenie mówiące o istnieniu rozwiązań takich problemów $LP$:

![image.png](projekt_alg_Simplex_zdj/simplex_tw1_wyklad.png)

Idea algorytmu sympleks polega innymi słowy na tym, żeby "stopniowo poprawiać" rozwiązanie dopuszczalne (problemu $LP$ w postaci kanonicznej). 

To "stopniowe poprawianie" polegać będzie na przechodzeniu między sąsiednimi wierzchołkami. 

Przejście od jednego wierzchołka do drugiego sąsiedniego wierzchołka (lub równoważnie: przejście od jednego bazowego rozwiązania dopuszczalnego do drugiego sąsiedniego) będzie polegało na wymianie jednej zmiennej (kolumny) bazowej.

Wykonując takie przejścia automatycznie dbamy cały czas o zachowanie dopuszczalności rozwiązania.

Algorytm kończy działanie, jeśli aktualne bazowe rozwiązanie dopuszczalne jest optymalne.

![image.png](projekt_alg_Simplex_zdj/wieloscian_alg_simplex.png)

Rys. 2. Wielościan algorytmu sympleksowego w trzech wymiarach (źródło: https://upload.wikimedia.org/wikipedia/commons/d/d4/Simplex-method-3-dimensions.png).

---

Żeby nie tłumaczyć wszystkiego tak na "sucho", będziemy dalej mieszać opis teoretyczny oraz implementację, "przeplatając" je z kolejnymi etapami rozwiązywania następującego zadania.

`Zadanie:`

Pewna firma zajmuje się produkcją dwóch rodzajów lodów:

- L1 – lody waniliowe,

- L2 – lody czekoladowe.

Proces produkcyjny obejmuje etapy takie jak mieszanie, pasteryzacja i zamrażanie. 

Produkcja przebiega w sposób ciągły – masa lodowa przepływa przez maszyny i może być dozowana w dowolnych ilościach. 

Maszyny można w każdej chwili uruchamiać i zatrzymywać, a partie produkcyjne można dzielić na dowolne porcje. Dlatego przyjmujemy, że:

- jednostką produkcyjną (detalem) jest 1 litr gotowych lodów,

- zmienne decyzyjne (czyli liczba litrów danego rodzaju lodów produkowanych tygodniowo) mogą przyjmować dowolne nieujemne wartości rzeczywiste.

Dla każdego rodzaju lodów znane są:

- czasy pracy potrzebne na każdej z maszyn (w minutach na 1 litr lodów),

- maksymalna dostępność czasowa każdej maszyny w tygodniu (w minutach),

- zysk jednostkowy ze sprzedaży 1 litra danego rodzaju lodów (w zł).

Dane te przedstawione zostały w poniższej tabeli:

<img src="projekt_alg_Simplex_zdj/simplex_zadanie_tabelka_dane_sample2.png" width="1100">

`Polecenia do wykonania:`

a) Zdefiniuj zmienne decyzyjne, sformułuj funkcję celu oraz wszystkie ograniczenia, tworząc kompletny model programowania liniowego.

b) Przekształć model do postaci kanonicznej (standardowej) – zapisz wektor funkcji celu, macierz technologiczną $A$ oraz wektor ograniczeń $b$.

c) Zastosuj metodę Simplex, aby obliczyć optymalne wartości zmiennych decyzyjnych.

---
`Rozwiązania:`

a) Przyjmijmy następujące oznaczenia na zmienne decyzyjne:

- $x_1$ - liczba litrów lodów L1 (waniliowych) produkowanych tygodniowo

- $x_2$ - liczba litrów lodów L2 (czekoladowych) produkowanych tygodniowo

Należy rozwiązać następujący problem optymalizacyjny:

$\begin{cases} 40 \cdot x_1 + 50 \cdot x_2 \to \text{max} \\ \text{przy ograniczeniach:} \\ \quad \, \, \, \, \qquad \qquad \quad \, \, x_1, x_2 \ge 0 \qquad \quad \text{ilość l wyprodukowanych lodów nie może być ujemna} \\ \begin{aligned} & 10 \cdot x_1 & + \, \, & 20 \cdot x_2 & \le & \, \, 3500 \quad &\text{ograniczenie na czas działania mieszalnika} & \, \\  & 40 \cdot x_1 & + \, \, & 30 \cdot x_2 & \le & \, \, 980\text{ }  &  \text{ograniczenie na czas działania pasteryzatora} & \, \\  & 100 \cdot x_1 & + \, \,  & 200 \cdot x_2 & \le & \, \, 5600 & \text{ograniczenie na czas działania zamrażarki} & \, \end{aligned} \end{cases}$

b) Aby zapisać powyższy problem optymalizacyjny w postaci kanonicznej (nazywanej również standardową lub bazową), wprowadzamy tzw. zmienne zapasu (ang. slack variables). 

Zmienne te pozwalają przekształcić każde z ograniczeń typu „≤” w równoważne ograniczenie równościowe, co jest wymagane do zastosowania metody Simplex.

W naszym przypadku wprowadzamy trzy zmienne zapasu — po jednej dla każdego z ograniczeń odpowiadających tygodniowym zasobom czasowym trzech maszyn. 

Oznaczamy je odpowiednio jako: $s_1, s_2, s_3$, gdzie każda z nich reprezentuje niewykorzystany tygodniowy zasób czasowy danej maszyny. Czyli:

- $s_1$: niewykorzystowany czas mieszalnika,

- $s_2$: niewykorzystowany czas pasteryzatora,

- $s_3$: niewykorzystowany czas zamrażarki.

Po dodaniu zmiennych zapasu, problem w postaci standardowej przyjmuje postać:

$\begin{cases} 40 \cdot x_1 + 50 \cdot x_2 + 0 \cdot s_1 + 0 \cdot s_2 + 0 \cdot s_3 \to \text{max} \\ \text{przy ograniczeniach:} \\ \quad \, \, \, \, \quad \qquad \qquad \qquad \qquad \qquad \qquad \, \, \, \, \, x_1, x_2, s_1, s_2, s_3 \ge 0 \\ \begin{aligned} & 10 \cdot x_1 & + \, \, & 20 \cdot x_2 & + \, \, & 1 \cdot s_1 & + \, \, & 0 \cdot s_2 & + \, \, & 0 \cdot s_3 & \le &\, \, 3500 \\  & 40 \cdot x_1 & + \, \, & 30 \cdot x_2 & + \, \, & 0 \cdot s_1 & + \, \, & 1 \cdot s_2 & + \, \, & 0 \cdot s_3 & \le &\, \, 980\\  & 100 \cdot x_1 & + \, \,  & 200 \cdot x_2 & + \, \, & 0 \cdot s_1 & + \, \, & 0 \cdot s_2 & + \, \, & 1 \cdot s_3 & \le & \, \, 5600 \end{aligned} \end{cases}$

Oznaczmy $n = 2$ (liczba zmiennych oryginalnie). Oznaczmy $m = 3$ (liczba ograniczeń, tzn. maszyn, bo mamy jedno ograniczenie dla każdej z trzech maszyn). Oczywiście więc $m < m+n$ (istotnie: $3 < 5$).

Wektor zmiennych decyzyjnych (bez zmiennych sztucznych) jest postaci: $\bf{x} = \left[ \begin{aligned} x_1 \\ x_2 \end{aligned} \right] \in \mathbb{R}^n,$, wektor zmiennych sztucznych jest postaci: $\bf{s} = \left[ \begin{aligned} s_1 \\ s_2 \\ s_3 \end{aligned} \right] \in \mathbb{R}^m,$, natomiast ze zmiennymi sztucznymi jest postaci: $\hat{\bf{x}} = \left[ \begin{aligned} x_1 \\ x_2 \\ s_1 \\ s_2 \\ s_3 \end{aligned} \right] \in \mathbb{R}^{n+m}.$

Przyjmijmy oznaczenie: $\bf{c} = \left[ \begin{aligned} &\, 40 \\ &\, 50 \end{aligned} \hspace{0.1 em} \right] \in \mathbb{R}^{n}.$ oraz: $\hat{\bf{c}} = \left[ \begin{aligned} &40 \\ &50 \\ &\, 0 \\ & \, 0 \\ &\, 0 \end{aligned} \right] \in \mathbb{R}^{n+m}.$

A więc funkcja celu (dla oryginalnego problemu nierownościowego w postaci standardowej) zdefiniowana jest w sposób następujący: $$\mathop{\forall} \left( {\bf{x}} = (x_1 \, \, \, \, x_2)^T \in \R^{n} \right) \, \, \, \, f(\bf{x}) = \bf{c}^T \bf{x} = [40 \quad 50] \cdot \left[ \begin{aligned} x_1 \\ x_2 \end{aligned} \right] = 40 \cdot x_1 + 50 \cdot x_2.$$ 

Natomiast funkcja celu (dla problemu równościowego, tzn. oryginalnego nierownościowego przekształconego z postaci standardowej do postaci kanonicznej) zdefiniowana jest w sposób następujący: $$\mathop{\forall} \left( \hat{\bf{x}} = (x_1 \, \, \, \, x_2 \, \, \, \, s_1 \, \, \, \, s_2 \, \, \, \, s_3)^T \in \R^{n+m} \right) \, \, \, \, \hat{f}(\hat{\bf{x}}) = \hat{\bf{c}}^T \hat{\bf{x}} = [40 \quad 50 \quad 0 \quad 0 \quad 0] \cdot \left[ \begin{aligned} x_1 \\ x_2 \\ s_1 \\ s_2 \\ s_3 \end{aligned} \right] = 40 \cdot x_1 + 50 \cdot x_2 + 0 \cdot s_1 + 0 \cdot s_2 + 0 \cdot s_3.$$ 

Dalej, przyjmujemy oznaczenia na:
- (oryginalną) macierz (technologiczną?) dla problemu nierównościowego (w postaci standardowej): $A = \left[ \begin{aligned} & \, \, 10  & 20\\ &\, \, 40 &30\\ &100 \, & 200 \end{aligned} \right] \in \mathcal{M}_{m \times n}$

- macierz (technologiczną?) dla problemu równościowego (oryginalnego ale w postaci kanonicznej): $\hat{A} = [A \mid I_3] = \left[ \begin{aligned} & \, \, 10  & 20 \quad \, \, & 1 \, \, & 0 \quad & 0 \\ &\, \, 40 & 30 \quad \, \, & 0 \, \,  & 1 \quad & 0\\ &100 \, & 200 \quad \, \, & 0 \, \, & 0 \quad & 1 \end{aligned} \right] \in \mathcal{M}_{m \times (n+m)}$

- wektor ograniczeń: $\qquad \, b =  \left[ \, \begin{aligned} &3500 \\ &\, 980 \\ &5600 \end{aligned} \, \right] \in \mathbb{R}^m$

Nasz problem optymalizacyjny (problem $LP$) najpierw więc przedstawiliśmy (w punkcie a) w postaci standardowej (nierównościowej):

$$\begin{cases} \text{zmaksymalizować:} \quad \bf{c}^T \bf{x} \\ \text{przy warunkach:} \quad \, \, \, \, A \cdot \bf{x} \le \bf{b} \\ \qquad \qquad \qquad \quad \quad \, \, \, \, \quad \, \, \, \, \bf{x} \ge \theta_{\mathbb{R}^n} \end{cases}$$

a teraz równoważnie przedstawiliśmy go w postaci kanonicznej (równościowej):

$$\begin{cases} \text{zmaksymalizować:} \quad \hat{\bf{c}}^T \hat{\bf{x}} \\ \text{przy warunkach:} \quad \, \, \, \, \hat{A} \cdot \hat{\bf{x}} = \bf{b} \\ \qquad \qquad \qquad \quad \quad \, \, \, \, \quad \, \, \, \, \hat{\bf{x}} \ge \theta_{\mathbb{R}^{n+m}} \end{cases}$$

---

Opieramy się teraz merytorycznie na [1], jednak przy oznaczeniach pozostajemy bardziej "wierni" wykładowi [3]. Oczywiście opieramy się też na oznaczeniach przyjętych powyżej w tym tekście.

Załóżmy, że $\bf{x}$ jest bazowym rozwiązaniem dopuszczalnym powyższego problemu $LP$ przekształconego do postaci kanonicznej (jak w podpunkcie b), przy czym $\bf{x}^T = [x_B \quad x_N]^T$, gdzie $\bf{x}_B \ge \bf{0}$ oraz $\bf{x}_N = \bf{0}$ oraz $\bf{x}_B \in \mathbb{R}^m$ oraz $\bf{x}_N \in \mathbb{R}^n$.

Wówczas macierz $\hat{A}$ da się przedstawić w postaci sklejenia macierzy $\hat{A} = [B \quad N]$, gdzie $B \in \mathcal{M}_{m \times m}$ jest nieosobliwa (a więc równoważnie: $\mathop{\text{rank}} B = m$) oraz $N \in \mathcal{M}_{m \times n}$.

Macierze $B, N$ za pomocą ich kolumn będziemy zapisywać następująco: $B = [B_1, \ldots, B_m], N = [N_{m+1}, \ldots, N_{m+n}]$.

Przejście od bazowego rozwiązania dopuszczalnego $\bf{x}$ (bazy dopuszczalnej $B$) do sąsiedniego bazowego rozwiązania dopuszczalnego $\bf{\bar{x}}$ (sąsiedniej bazy dopuszczalnej $\bar{B}$) polega na "wyjściu z bazy" pewnej bazowej kolumny $B_{i_{\star}}$ (zmiennej bazowej $x_{i_{\star}}$), dla pewnego $i_{\star} \in \{ 1, \ldots, m\}$, oraz "wejściu do bazy" pewnej niebazowej kolumny $N_k$ (zmiennej niebazowej $x_k$), dla pewnego $k \in \{ m+1, \ldots, m+n\}$. 

Konkretnie to przejście polega więc na wymianie jednej zmiennej (kolumny) między zbiorem kolumn bazowych oraz zbiorem kolumn niebazowych. Przejście to wygląda w ten sposób:

$$ \bf{x}^T = [x_1^B, \ldots, x_{i_{\star}-1}^B, x_{i_{\star}}^B, x_{i_{\star}+1}^B, \ldots, x_m^B, \bf{0}_{n}] \quad \to \quad \bar{\bf{x}}^T = [x_1^B, \ldots, x_{i_{\star}-1}^B, x_{i_{\star}+1}^B, \ldots, x_m^B, x_k^N, \bf{0}_{n-1}] $$

W "języku" macierzy bazowych to przejście wygląda tak:

$$ B = [B_1, \ldots, B_{i_{\star}-1}, B_{i_{\star}}, B_{i_{\star} + 1}, \ldots, B_m] \quad \to \quad  \bar{B} = [B_1, \ldots, B_{i_{\star}-1}, B_{i_{\star} + 1}, \ldots, B_m, N_k]$$

W kontekście algorytmu Simplex bardzo ważne są:

1. kryteria wyjścia, tzn. kryteria doboru zmiennej (kolumny) bazowej, która wychodzi z bazy

2. kryteria wejścia, tzn. kryteria doboru zmiennej (kolumny) niebazowej, która wchodzi do bazy

Oznaczmy indeks zmiennej (kolumny), która wchodzi do bazy, jako $k$ - oczywiście $k \in \{ m + 1, \ldots, m+n\}$.

Niech $\bf{y}_k := B^{-1} N_k$ (wówczas $\bf{y}_k \in \mathbb{R}^m$) - wówczas $\bf{y}_k$ jest przedstawieniem kolumny niebazowej $N_k$ w aktualnej bazie $B$. Da się udowodnić następujące twierdzenie.

`Twierdzenie 1 (Kryterium wyjścia)`: Jeżeli w wektorze $\bf{y}_k$ przynajmniej jedna składowa jest dodatnia oraz:

$$ \bar{\Theta}_k := \frac{x_{i_{\star}}^B}{y_{i_{\star}}^k} = \mathop{\text{min}} \left\{ \frac{x_{i}^B}{y_{i}^k} \colon y_i^k > 0, 1 \le i \le m \right\}$$

to $\bf{\bar{x}}$ jest sąsiednim bazowym rozwiązaniem dopuszczalnym, gdzie:

$$
\bar{\mathbf{x}} = \bar{\mathbf{x}}(\overline{\Theta}_k) =
\left[
\begin{array}{c}
x_1^B - \bar{\Theta}_k y_1^k \\
\vdots \\
x_{i^*-1}^B - \bar{\Theta}_k y_{i^*-1}^k \\
0 \\
x_{i^*+1}^B - \bar{\Theta}_k y_{i^*+1}^k \\
\vdots \\
x_m^B - \bar{\Theta}_k y_m^k \\
\bf{0} \\
\bar{\Theta}_k \\
\bf{0}
\end{array}
\right]
$$

Ponadto, jeżeli żadna składowa wektora $\bf{y}_k$ nie jest dodatnia, wówczas zbiór rozwiązań dopuszczalnych jest nieograniczony.

Algorytm Simplex w takim przypadku wykrywa to i kończy działanie, "wyrzucając" odpowiedni komunikat. Czyli algorytm ten jest w stanie, przy konstrukcji sąsiedniego bazowego rozwiązania dopuszczalnego, stwierdzić, czy wartość funkcji celu jest ograniczona z góry na zbiorze rozwiązań dopuszczalnych.

Przyjmijmy oznaczenie: $\bf{c}_B := [c_1, \ldots, c_m]^T$ oraz $z_k := (\bf{c}_B)^T {\bf{y}}_k = (\bf{c}_B)^T B^{-1} N_k = (\bf{c}_B)^T B^{-1} A_k$.

Jeśliby wprowadzić oznaczenie $\lambda := (B^{-1})^T c_B$, to wówczas $\lambda^T = [(B^{-1})^T c_B]^T = c_B^T B^{-1}$. Wówczas $z_k = \lambda^T A_k$.

Przy tych oznaczeniach, wektor $\lambda$ nazywamy $\emph{wektorem mnożników sympleksowych}$.

Przy jednym takim przejściu, wartość funkcji celu zmienia się o $\bar{\Theta}_k \cdot (c_k-z_k)$. Stąd, jeśli $c_k - z_k > 0$, to takie przejście prowadzi do poprawy wartości funkcji celu przy takim przejściu. 

A więc jeżeli istnieje kolumna niebazowa $N_k$, dla pewnego $k \in \{ m + 1, \ldots, m + n\}$, dla której $c_k - z_k > 0$, to aktualne bazowe rozwiązanie dopuszczalne $\bf{x}$ nie jest optymalne. Stąd:

`Twierdzenie 2 (Kryterium wejścia)`: Indeks zmiennej (kolumny) niebazowej $k$, wchodzącej do bazy, może być wyznaczony w sposób zachłanny, tzn.

$$ c_k - z_k = \mathop{\text{max}} \{ c_j - z_j \colon c_j - z_j > 0, m + 1 \le j \le m + n\} $$

tyle że jest to metoda dość droga obliczeniowo. Taniej jest wybrać pierwszą kolumnę niebazową $j$, dla której $c_j - z_j > 0$. Ponadto okazuje się, że zachodzi:

`Twierdzenie 3 (Kryterium optymalności)`: Jeżeli dla wszystkich kolumn niebazowych $N_k$, gdzie $k \in \{ m+1, \ldots, m+n\}$, zachodzą nierówności $c_k - z_k \le 0$, to aktualny $\bf{x}$ jest optymalnym rozwiązaniem rozważanego $LP$.

Ponadto zachodzi twierdzenie:

![image.png](projekt_alg_Simplex_zdj/simplex_tw2_wyklad.png)

Warto jednak wspomnieć, że to twierdzenie ma jedno istotne założenie. Otóż musimy w specyficzny sposób wybierać indeksy kolumn wychodzących z bazy i wchodzących do bazy. Bardziej formalnie, to twierdzenie brzmiałoby tak (formułujemy je tak jak było ono sformułowane w źródle [1]):

`Twierdzenie 4 (Algorytm Simplex zatrzymuje się)` Załóżmy, że $N_k$ będzie kolumną wchodzącą do bazy wybraną następująco:

$$ k = \mathop{\text{min}} \{ j \colon c_j - z_j > 0, m + 1 \le j \le m + n\}$$

oraz w przypadku pojawienia się niejednoznaczności przy zastosowaniu kryterium wyjścia, przyjmiemy jako indeks zmiennej wychodzącej z bazy indeks $i_{\star}$ będący indeksem o najmniejszej wartości spośród indeksów wyznaczających minimalną wartość ilorazu $\frac{x_{i_\star}^B}{y_{i_\star}^k}$. Wówczas algorytm sympleks skończy działanie w skończonej liczbie kroków.

---

Jako pierwszą implementujemy funkcję, która pozwala, za pomocą poniżej własnoręcznie zaimplementowanego solvera dla algorytmu Simplex, rozwiązywać problemy $LP$ polegające na maksymalizacji $\bf{c}^T \bf{x}$ przy ograniczeniach $\bf{x} \ge 0$ oraz ograniczeniach mieszanych typów, tzn. ogólnie mogących przyjąć formy:

- nierówności typu "$\le$": $\quad A_{le} \cdot \bf{x} \le b_{le}$

- nierówności typu "$\ge$": $\quad A_{ge} \cdot \bf{x} \ge b_{ge}$

- równości: $\qquad \qquad \quad \, \, \, \, A_{eq} \cdot \bf{x} = b_{eq}$

Funkcja ta zbierać będzie wszystkie te ograniczenia mieszanych typów i doprowadzać je do jednolitej postaci $A \le \bf{x} \le b$, uwzględniającej wszystkie te ograniczenia jednocześnie.

Z tej funkcji będziemy oczywiście w bardzo kluczowych "momentach" kodu korzystać - odgrywa ona dość istotną rolę, ponieważ dzięki niej możliwości solvera własnoręcznie zaimplementowanego są dużo większe niż tylko typowo rozwiązywanie problemów $LP$ z ograniczeniami wyłącznie w postaci równości (wtedy mówimy o postaci kanonicznej) czy też nierówności typu $\le$.

In [2]:
# --- Narzędzie: przetwarzanie ogólnych ograniczeń LP ---
def preprocess_general_lp(A_le=None, b_le=None, A_ge=None, b_ge=None, A_eq=None, b_eq=None, return_origin=False):
    """
    Funkcja przyjmuje:
    - macierz A_le i wektor b_le  → ograniczenia typu A_le * x ≤ b_le
    - macierz A_ge i wektor b_ge  → ograniczenia typu A_ge * x ≥ b_ge
    - macierz A_eq i wektor b_eq  → ograniczenia typu A_eq * x = b_eq

    Zwraca (domyślnie) macierz A oraz wektor b, które łącznie reprezentują wszystkie
    ograniczenia w postaci A * x ≤ b. Jeśli return_origin=True, dodatkowo zwraca
    listę origin_list, gdzie każdy element to etykieta wskazująca, z jakiego
    typu ograniczenia pochodzi dany wiersz:
      - 'le'  → wiersz oryginalnie z A_le x ≤ b_le
      - 'ge'  → wiersz pierwotnie z A_ge x ≥ b_ge (po przekształceniu przez −1)
      - 'eq'  → wiersz pochodzący z rozbicia A_eq x = b_eq na dwie nierówności

    Umożliwia wygodne wykorzystanie w dalszych etapach (np. w fazie I Simplexu),
    by wiedzieć, które wiersze wolno dodatkowo odwrócić, a których nie.
    """
    A_rows, b_rows = [], []
    origin_list = []

    # 1) Ograniczenia "≤" (le)
    if A_le is not None:
        for Ai, bi in zip(np.atleast_2d(A_le), b_le):
            A_rows.append(Ai.copy())
            b_rows.append(bi)
            origin_list.append('le')

    # 2) Ograniczenia "≥" (ge) → zamiana na "≤" przez pomnożenie przez -1
    if A_ge is not None:
        for Ai, bi in zip(np.atleast_2d(A_ge), b_ge):
            A_rows.append(-Ai.copy())   # -A_ge
            b_rows.append(-bi)          # -b_ge
            origin_list.append('ge')

    # 3) Ograniczenia "=" (eq) → rozbicie na dwie nierówności "≤"
    if A_eq is not None:
        for Ai, bi in zip(np.atleast_2d(A_eq), b_eq):
            # A_eq x ≤ b_eq
            A_rows.append(Ai.copy())
            b_rows.append(bi)
            origin_list.append('eq')
            # -A_eq x ≤ -b_eq
            A_rows.append(-Ai.copy())
            b_rows.append(-bi)
            origin_list.append('eq')

    # 4) Scalanie wierszy w macierz A i wektor b
    A = np.vstack(A_rows)      # (liczba_wierszy × n)
    b = np.hstack(b_rows)      # (liczba_wierszy,)

    if return_origin:
        return A, b, origin_list
    else:
        return A, b

Dane dla naszego problemu:

In [3]:
A = np.array([[10,20],
              [40,30],
              [100,200]]).astype(float)
m, n = A.shape
b = np.array([[3500],
              [980],
              [5600]]).astype(float)
c = np.array([[40],
              [50]]).astype(float)

In [4]:
m #liczba wierszy "oryginalnej" macierzy A (czyli dla LP w postaci standardowej, tzn. nierównościowej)

3

In [5]:
n #liczba kolumn "oryginalnej" macierzy A (czyli dla LP w postaci standardowej, tzn. nierównościowej)

2

Sprawdźmy, co robi ta funkcja.

In [6]:
A_ub, b_ub, origin_list = preprocess_general_lp(A_le=A, b_le=b, return_origin=True)

In [7]:
A_ub

array([[ 10.,  20.],
       [ 40.,  30.],
       [100., 200.]])

In [8]:
A_ub.shape

(3, 2)

In [9]:
b_ub

array([3500.,  980., 5600.])

In [10]:
b_ub.shape

(3,)

In [11]:
origin_list

['le', 'le', 'le']

In [12]:
#Jak działa np.atleast_2d w połączeniu z zip w kodzie powyżej?
for Ai, bi in zip(np.atleast_2d(A),b): #--> nawet fajne :)
    print("Ai =", Ai)
    print("bi =", bi)
    print()

Ai = [10. 20.]
bi = [3500.]

Ai = [40. 30.]
bi = [980.]

Ai = [100. 200.]
bi = [5600.]



Podsumowując powyższe, algorytm sympleks wyznaczający optymalną wartość funkcji celu w problemie $LP$ sformułowanym w postaci kanonicznej da się wyrazić za pommocą następującego pseudokodu (który został przedstawiony na wykładzie [3]):

![image.png](projekt_alg_Simplex_zdj/simplex_pseudokod1.png)

Zrobimy to w ramach funkcji `simplex_core`. Będzie to funkcja pobierająca macierze/wektory $\hat{A}, b, \hat{\bf{c}}$ dla problemu $LP$ zapisanego już w postaci równościowej (kanonicznej), a więc po ewentualnym (jeżeli koniecznym) "dołożeniu" slacków/zmiennych sztucznych. Więcej o tym, co to są "slacki" i zmienne sztuczne i czym to się różni, trochę później (niżej). Przykładowo, według notacji powyższej, jeśli jedynym co dokładamy jest "pakiet" m "slacków", to ta funkcja rozwiązuje właśnie problem:

$$\begin{cases} \text{zmaksymalizować:} \quad \hat{\bf{c}}^T \hat{\bf{x}} \\ \text{przy warunkach:} \quad \, \, \, \, \hat{A} \cdot \hat{\bf{x}} = \bf{b} \\ \qquad \qquad \qquad \quad \quad \, \, \, \, \quad \, \, \, \, \bf{x} \ge \theta_{\mathbb{R}^{n+m}} \end{cases}$$

W istocie, będzie to funkcja pomocnicza dla takiej "ostatecznej" (końcowej) funkcji (o nazwie `simplex_solve_general`), która będzie rozwiązywała wyjściowy, uniwersalnie zapisany nierównościowy problem $LP$ (tzn. podany w postaci standardowej wg oznaczeń powyżej, ogólnie z ograniczeniami już po "przepuszczeniu" przez funkcję `preprocess_general_lp`).

Opisem kroku (1) - i tego jak znajdywać tą początkową bazę - zajmiemy się troszeczkę później.

In [13]:
# --- Rdzeń Algorytmu Simplex (Faza II) ---
def simplex_core(A_hat, b, c_hat, B_cols = None, return_more=False, tol=1e-9, draw_tableaus=False):
    """
    Ta funkcja jest implementacją powyżej przedstawionego pseudokodu (z wykładu), realizującą fazę II algorytu Simplex.
    Więcej o tym, że algorytm Simplex w ogólności implementuje się dwu-fazowo, później.

    Ta funkcja przyjmuje następujące argumenty (według notacji bezpośrednio powyżej, z postaci kanonicznej):
    - A_hat: macierz lewych stron ograniczeń (m x n_hat)
    - b: wektor prawych stron ograniczeń (m x 1)
    - c_hat: wektor współczynników celu (n_hat x 1)
    - B_cols: lista indeksów kolumn bazy --> TO JEST WAŻNE! WYWOŁUJĄC TĄ FUNKCJĘ JUŻ MUSIMY WIEDZIEĆ, KTÓRE KOLUMNY MACIERZY a_hat TWORZĄ BAZĘ (MACIERZ BAZOWĄ B)
    - return_more: jeśli True, zwracamy również wartość funkcji celu i końcową bazę (w sensie numerów kolumn)
    - tol: tolerancja dla sprawdzania optymalności i unikania dzielenia przez 0 - zmienna typowo do "czysto technicznych celów obliczeń numerycznych"
    """
    m, n_hat = A_hat.shape

    # Inicjalizujemy macierz bazową (bazę)
    if B_cols is None:
        found_basis = False #flaga pomocnicza
        for candidate in combinations(range(n_hat), m):
            M_candidate = A_hat[:, candidate]
            # Sprawdzamy, czy macierz kandydata ma pełną rangę
            if np.linalg.matrix_rank(M_candidate) < m:
                continue
            # Rozwiązujemy układ M_candidate * x_B = b
            xB_candidate = np.linalg.solve(M_candidate, b)
            # Uznajemy bazę, gdy rozwiązanie jest dopuszczalne (każdy element >= -tol)
            if np.all(xB_candidate >= -tol):
                B = list(candidate)
                found_basis = True
                break
        if not found_basis:
            raise Exception("Nie znaleziono poprawnej bazy, która daje dopuszczalne rozwiązanie.")
    else:
        B = list(B_cols)
    
    # Inicjalizujemy wektor x (n_hat x 1) jako wektor zer
    x = np.zeros((n_hat, 1))

    iteration = 0
    while True:
        # 1) Rozwiązanie układu B x_B = b, gdzie B = A_hat[:, B], tzn. wyznaczenie rozwiązania bazowego x_B --> nie odwracamy macierzy bazowej wprost dla stabilności numerycznej
        #Z pseudokodu powyżej krok nr 1 --> dana jest baza B, że x_B = B^-1 * b >= 0
        M = A_hat[:, B]                        # macierz bazowa (m x m)
        xB = np.linalg.solve(M, b).flatten()   # rozwiązanie x_B (flattowany na wektor długości m)

        # 2) Krok typowo programistyczny - musimy "podmienić" to rozwiązanie bazowe a na pozostałych pozycjach wstawić zera
        x.fill(0)
        for i, j in enumerate(B):
            x[j, 0] = xB[i]                   # wstawiamy xB na pozycje odpowiadające indeksom bazy

        # 3) Obliczenie wektora mnożników sympleksowych λ: (B^T) λ = c_B
        #Z pseudokodu powyżej krok nr 2 --> wyznaczyć wektor mnożników sympleksowych
        cB = c_hat[B].reshape((m, 1))         # wektor c dla zmiennych bazowych (m x 1)
        lamb = np.linalg.solve(M.T, cB).flatten()

        # --- Jeżeli draw_tableaus=True, budujemy i drukujemy tablicę ---
        if draw_tableaus:
            # Obliczamy z_j i c_j - z_j
            z_j = (lamb @ A_hat).flatten()
            c_vec = c_hat.flatten()
            cj_minus_zj = c_vec - z_j

            header = f"\nIteracja {iteration}"
            print(header)
            print("-" * (8 * (n_hat + 3)))

            # Wiersz c_j
            line_cj = "    cj  |"
            for j in range(n_hat):
                line_cj += f" {c_vec[j]:>6.2f} "
            print(line_cj)

            # Wiersz nagłówków kolumn zmiennych + "RHS"
            line_vars = "  j    |"
            for j in range(n_hat):
                line_vars += f"  x{j:<4}"
            line_vars += "  RHS   |"     
            print(line_vars)
            print("-" * (8 * (n_hat + 3)))

            # Wiersze odpowiadające zmiennym bazowym
            B_inv = np.linalg.inv(M)
            BA = B_inv @ A_hat
            for i_row, j_base in enumerate(B):
                row_coeffs = BA[i_row, :]
                line = f"  {j_base:>3d}  |"
                for j in range(n_hat):
                    line += f" {row_coeffs[j]:>6.2f}"
                line += f" | {xB[i_row]:>8.2f}"  # to jest właśnie RHS
                print(line)

            # Wiersz z_j
            print("-" * (8 * (n_hat + 3)))
            line_zj = "  z_j  |"
            for j in range(n_hat):
                line_zj += f" {z_j[j]:>6.2f}"
            line_zj += " |"
            print(line_zj)

            # Wiersz c_j - z_j
            line_cj_zj = "c_j–z_j |"
            for j in range(n_hat):
                line_cj_zj += f" {cj_minus_zj[j]:>6.2f}"
            line_cj_zj += " |"
            print(line_cj_zj)
            print("-" * (8 * (n_hat + 3)))
            # ===== koniec rysowania tabeli =====

        #Z pseudokodu powyżej krok nr 3 --> wybrać z N taką kolumnę A_k, że p_k = c_k - lamb^T * A_k > 0. Jeśli nie ma, to znaleziono rozwiązanie optymalne
        # 4) Obliczenie wskaźników p: p = c_N - λ^T A_N --> robimy to jednak sprytnie, bo za pomocą wektoryzacji
        N = [j for j in range(n_hat) if j not in B] # Lista indeksów zmiennych niebazowych
        A_N = A_hat[:, N]                     # macierz kolumn niebazowych (m x |N|)
        cN = c_hat[N].flatten()               # wektor c dla kolumn niebazowych
        p = cN - lamb @ A_N                   # wektor kosztów redukowanych (|N|-wymiarowy)

        # 5) Sprawdzamy warunek optymalności: jeśli nie ma p[i] > tol, znaleźliśmy szukane ekstremum funkcji celu
        enters = [N[i] for i in range(len(p)) if p[i] > tol]
        if not enters:
            # brak zmiennych z dodatnim wskaźnikiem p → optymalność
            x_opt = x.flatten()
            if return_more:
                # obliczamy wartość funkcji celu f_opt = c_hat^T x
                f_opt = float((c_hat.T @ x).item())
                return x_opt, f_opt, sorted([int(elem) for elem in B])         # zwracamy rozwiązanie, wartość celu i bazę (indeksy, tzn. numery, kolumn z których składa się baza)
            return x_opt                       # zwracamy tylko rozwiązanie x

        # 6) Wybór zmiennej wchodzącej do bazy (zasada Blooma/Blanda: najmniejszy indeks)
        e = min(enters)

        #Z pseudokodu powyżej krok nr 4 --> wyznaczyć y = B^-1 * A_k
        # 7) Obliczenie kierunku dopuszczalnego w przestrzeni zmiennych bazowych y = B^{-1} A[:, e]
        y = np.linalg.solve(M, A_hat[:, e].reshape((m, 1))).flatten()

        # 8) Test ilorazowy: wybieramy minimalne θ = min{xB[i]/y[i] dla y[i] > tol, i <= m}
        #    Jeśli dla wszystkich y[i] ≤ tol, LP jest nieograniczone w kierunku e
        ratios = [xB[i] / y[i] if y[i] > tol else np.inf for i in range(m)]
        theta = min(ratios)
        if theta == np.inf:
            # LP nieograniczone → brak ograniczenia od góry dla zmiennej e
            raise Exception("LP nieograniczone")

        # 9) Wybór zmiennej wychodzącej (Bland): jeśli kilka i ma xB[i]/y[i] = θ, wybieramy tę z najmniejszym indeksem B[i]
        leaves = [i for i, val in enumerate(ratios) if abs(val - theta) <= tol]
        leave = min(leaves, key=lambda i: B[i])
        # Zamieniamy w bazie zmienną z indeksem B[leave] na e
        B[leave] = e
        # Powtarzamy iterację, aktualizując bazę
        iteration += 1

Sprawdźmy na razie samodzielnie, jeszcze bez "pomocy" ostatecznej funkcji `simplex_solve_general`, jak wyglądałoby rozwiązanie naszego problemu. 

Musimy więc sami "wyprodukować" macierz $\hat{A} = [A \mid I_m]$ oraz wektor $\hat{\bf{c}} = [\bf{c} \quad 0_m]$.

In [14]:
A_hat = np.hstack([A_ub, np.eye(m)])
A_hat

array([[ 10.,  20.,   1.,   0.,   0.],
       [ 40.,  30.,   0.,   1.,   0.],
       [100., 200.,   0.,   0.,   1.]])

In [15]:
A_hat.shape #wymiary: m x n_hat (n_hat = n+m = 2 + 3 = 5)

(3, 5)

In [16]:
c_hat = np.vstack([c, np.zeros((m,1))])
c_hat

array([[40.],
       [50.],
       [ 0.],
       [ 0.],
       [ 0.]])

In [17]:
c_hat.shape #wymiary: n_hat x 1

(5, 1)

In [18]:
b

array([[3500.],
       [ 980.],
       [5600.]])

In [19]:
b.shape #wymiary: m x 1

(3, 1)

Przyjmujemy bazę:

In [20]:
B_cols = [2,3,4]

No to zobaczmy, jakie będzie rozwiązanie naszego problemu.

In [21]:
x_opt, f_opt, B_cols_opt = tuple(simplex_core(A_hat,b,c_hat,B_cols,return_more=True,tol=1e-10))
print('x_opt ze zmiennymi luzu =', x_opt)
print('x_opt bez zmiennych luzu = (x_1, x_2) =', x_opt[:2])
print('f_opt =', f_opt)
print('B_cols_opt =', B_cols_opt)

x_opt ze zmiennymi luzu = [   5.6   25.2 2940.     0.     0. ]
x_opt bez zmiennych luzu = (x_1, x_2) = [ 5.6 25.2]
f_opt = 1484.0
B_cols_opt = [0, 1, 2]


Czyli najbardziej opłaca się produkować tygodniowo 5.6l lodów waniliowych i 25.2l czekoladowych - wówczas zysk tygodniowy będzie największy i wyniesie 1484 zł.

Zobaczmy porównanie wyników naszego solvera z solverem w bibliotece scipy.

In [22]:
res = linprog(c=-c, A_ub=A_ub, b_ub=b_ub,
                bounds=[(0, None)] * c.shape[0], method='highs')
print('scipy solver linprog - rozwiązanie: {},               wartość funkcji celu: {}'.format(res.x, -res.fun))
# Uwaga: SciPy dla maksymalizacji przyjmuje -c i zwraca -fun jako wartość celu.

scipy solver linprog - rozwiązanie: [ 5.6 25.2],               wartość funkcji celu: 1484.0


Yeey! Zgadza się! Wychodzi idealnie to samo :)

Nasza funkcja `simplex_core` ma jeszcze "umiejętność rysowania tabelek sympleksowych krok po kroku:

In [23]:
x_opt, f_opt, B_cols_opt = tuple(simplex_core(A_hat,b,c_hat,B_cols,return_more=True,tol=1e-10, draw_tableaus=True))
print('x_opt ze zmiennymi luzu =', x_opt)
print('x_opt bez zmiennych luzu = (x_1, x_2) =', x_opt[:2])
print('f_opt =', f_opt)
print('B_cols_opt =', B_cols_opt)


Iteracja 0
----------------------------------------------------------------
    cj  |  40.00   50.00    0.00    0.00    0.00 
  j    |  x0     x1     x2     x3     x4     RHS   |
----------------------------------------------------------------
    2  |  10.00  20.00   1.00   0.00   0.00 |  3500.00
    3  |  40.00  30.00   0.00   1.00   0.00 |   980.00
    4  | 100.00 200.00   0.00   0.00   1.00 |  5600.00
----------------------------------------------------------------
  z_j  |   0.00   0.00   0.00   0.00   0.00 |
c_j–z_j |  40.00  50.00   0.00   0.00   0.00 |
----------------------------------------------------------------

Iteracja 1
----------------------------------------------------------------
    cj  |  40.00   50.00    0.00    0.00    0.00 
  j    |  x0     x1     x2     x3     x4     RHS   |
----------------------------------------------------------------
    2  |   0.00  12.50   1.00  -0.25   0.00 |  3255.00
    0  |   1.00   0.75   0.00   0.03   0.00 |    24.50
    4  |   0

Autor poradnika [5] rozrysowuje przykład LP, pokazując właśnie kolejne kroki algorytmu Simplex poprzez rysowanie kolejnych tabelek Simplex:

![image.png](projekt_alg_Simplex_zdj/przyklad_yt.png)

![image.png](projekt_alg_Simplex_zdj/screen1_poradnik_white.png)

---

Pojawia się naturalne pytanie: w jaki sposób efektywnie wybrać kolumny tworzące początkową macierz bazową? Oczywiście można zastosować tzw. podejście brutalnej siły, polegające na sprawdzeniu wszystkich możliwych kombinacji $m$ kolumn spośród $(n + m)$ kolumn macierzy rozszerzonej $\hat{A}$ (gdzie $m$ i $n$ oznaczają odpowiednio liczbę równań i zmiennych decyzyjnych w pierwotnym problemie nierównościowym $LP$).

Jednakże takie rozwiązanie jest bardzo nieefektywne obliczeniowo – w najgorszym przypadku wymagałoby przeszukania ogromnej liczby kombinacji, co dla dużych i złożonych problemów skutkuje nieakceptowalnym czasem wykonania.

Na wykładzie [3] zaproponowano znacznie bardziej eleganckie i praktyczne podejście, pozwalające w sposób inteligentny i konstruktywny wyznaczyć pierwsze dopuszczalne bazowe rozwiązanie – a tym samym wybrać odpowiednie kolumny tworzące początkową macierz bazową $B$, od której można rozpocząć działanie algorytmu Simplex w kierunku znalezienia rozwiązania optymalnego. Poniżej przedstawiamy to podejście w formie pseudokodu.

![image.png](projekt_alg_Simplex_zdj/simplex_pseudokod2.png)

Możemy więc już zauważyć, że algorytm Simplex zwykle rozwiązuje problem programowania liniowego (podany w postaci nierównościowej) w dwóch etapach:

- Faza I – szukamy tzw. rozwiązania bazowego, czyli punktu startowego, od którego zaczniemy właściwe obliczenia. W praktyce sprowadza się to do wybrania odpowiednich kolumn, które utworzą tzw. macierz bazową. Ta faza także może być rozwiązana za pomocą funkcji `simplex_core`. W efekcie wyznaczamy to pierwsze bazowe rozwiązanie dopuszczalne lub stwierdzamy, że problem nie ma rozwiązania.

- Faza II – właściwe rozwiązywanie oryginalnego problemu, już przekształconego do postaci równościowej (kanonicznej). Tutaj też wykorzystujemy funkcję `simplex_core`.

Końcowa funkcja, czyli `simplex_solve_general`, będzie umiała:

- określić, które kolumny tworzą dobrą bazę (na podstawie wyniku z fazy I),

- przekształcić pierwotny problem (z ograniczeniami w postaci nierówności) do postaci kanonicznej (czyli z ograniczeniami jako równości), przez dodanie tzw. slacków (zmiennych uzupełniających) lub zmiennych sztucznych.

Zanim jednak zaimplementujemy tę główną funkcję, napiszemy jeszcze prostą funkcję pomocniczą, która przygotuje dane potrzebne do fazy I – zgodnie z tym, co pokazano na wcześniejszym slajdzie.

In [24]:
# --- Funkcja przygotowująca program do rozwiązania fazy I w algorytmie Simplex ---
def prepare_phase_I_general(A_le = None, b_le = None, A_ge=None, b_ge=None, A_eq=None, b_eq=None):
    """
    Ta funkcja, podobnie jak simplex_core, również jest funkcją pomocniczą, którą potem wykorzystuje metoda simplex_solve_general.
    To co robi ta funkcja, to zwraca dane w gotowej postaci pod fazę I realizacji algorytmu Simplex.
    W fazie I maksymalizujemy –∑(zmienne sztuczne), aby znaleźć dopuszczalne rozwiązanie bazowe x_B.

    Przyjmuje te same argumenty co preprocess_general_lp:
      A_le, b_le – ograniczenia A_le x ≤ b_le,
      A_ge, b_ge – ograniczenia A_ge x ≥ b_ge,
      A_eq, b_eq – ograniczenia A_eq x = b_eq.

    Zwraca:
      A_art   – macierz (m_all × (n_orig + 2·m_all)),
      b_mod   – wektor (m_all × 1),
      c_art   – wektor (n_orig + 2·m_all, 1),
      B0      – lista długości m_all z indeksami kolumn sztucznych,
      n_orig  – liczba kolumn w A_tmp --> A_tmp to macierz A zwracana przez preprocess_general_lp (po pełnej konwersji ≤, ≥, = na ≤), 
      m_all   – liczba wierszy w A_tmp
    """
    # 1) Zbieramy wszystkie nierówności w formie A_tmp x ≤ b_tmp. W tym celu wywołujemy preprocess_general_lp(…, return_origin=True),
    # dostając na wyjściu: A_tmp, b_tmp, origin_list. W A_tmp mamy wszystkie nierówności w formie „≤” (nawet te, które pierwotnie były „≥” lub „=”), 
    # natomiast b_tmp to odpowiadające prawe strony (mogą być ujemne lub dodatnie). 
    A_tmp, b_tmp, origin_list = preprocess_general_lp(
        A_le, b_le, A_ge, b_ge, A_eq, b_eq, return_origin=True
    )

    # 2) Nie ruszamy A_tmp ani b_tmp – tylko wyznaczamy z nich wymiary:
    m_all, n_orig = A_tmp.shape

    # 3) Konstruujemy I_slack = I_{m_all}  (zwykła macierz jednostkowa, bo każdą nierówność „≤” zamieniamy na równanie przez dodanie slacków),
    I_slack = np.eye(m_all)

    # 4) Budujemy macierz dla zmiennych sztucznych Ī: bierzemy macierz jednostkową i przekształcamy ją tak, że i-ty jej wiersz mnożyy przez –1 jeżeli b_tmp[i] < 0.
    rows_sign = np.where(b_tmp.flatten() >= 0, +1, -1)
    I_art_bar = np.diag(rows_sign) 

    # 5) Składamy pełne A_art = [ A_tmp | I_slack | I_art_bar ]
    A_art = np.hstack([A_tmp, I_slack, I_art_bar])
    # Czyli wymiary A_art:  m_all × (n_orig + m_all + m_all). 

    # 6) Wektor prawej strony fazy I to po prostu b_tmp w postaci kolumny
    b_mod = b_tmp.reshape(-1, 1)

    # 7) Konstruujemy wektor współczynników dla funkcji celu w fazie I
    # 7) najpierw n_orig zer (dla x), potem m_all zer (dla s, czyli slacków), potem m_all razy –1 (dla v, czyli zmiennych sztucznych)
    # Rrozwiązanie fazy I ma minimalizować ∑ v_i  (czyli maksymalizować –∑ v_i)
    c_art = np.vstack([
        np.zeros((n_orig + m_all, 1)),   # 0 dla [ x ; slack ]
        -np.ones((m_all, 1))             # –1 dla każdej sztucznej v_i
    ])

    # 8) Baza startowa B0 to wszystkie kolumny związane ze zmiennymi sztucznymi v_i, czyli te o indeksach od n_orig+m_all do n_orig+2*m_all–1
    B0 = list(range(n_orig + m_all, n_orig + 2 * m_all))

    return A_art, b_mod, c_art, B0, n_orig, m_all

Zobaczmy, w jaki sposób będą wyglądały nasze dane, przygotowane do wstawienia do `simplex_core`, aby rozwiązać fazę I.

In [25]:
A_art, b_mod, c_art, B0, n_orig, m_all = prepare_phase_I_general(A,b)

In [26]:
A_art

array([[ 10.,  20.,   1.,   0.,   0.,   1.,   0.,   0.],
       [ 40.,  30.,   0.,   1.,   0.,   0.,   1.,   0.],
       [100., 200.,   0.,   0.,   1.,   0.,   0.,   1.]])

In [27]:
b_mod

array([[3500.],
       [ 980.],
       [5600.]])

In [28]:
c_art

array([[ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [-1.],
       [-1.],
       [-1.]])

In [29]:
B0

[5, 6, 7]

In [30]:
n_orig

2

In [31]:
m_all

3

In [32]:
x1, f1, B1 = simplex_core(A_art, b_mod, c_art, B0, return_more=True)
if abs(f1) > 1e-9:
    raise Exception("Brak dopuszczalnego rozwiązania (faza I)")

In [33]:
x1

array([   5.6,   25.2, 2940. ,    0. ,    0. ,    0. ,    0. ,    0. ])

In [34]:
f1

0.0

In [35]:
B1 #kolumny, które należałoby użyć, aby skonstruować bazę

[0, 1, 2]

No to zaimplementujmy tą ostateczną funkcję `simplex_solve` i zobaczy, czy ta ostateczna funkcja radzi sobie z rozwiązaniem tego problemu, za pomocą jednego odwołania tylko do niej, czy robi sobie wszysko sama i sobie radzi, od A do Z.

In [36]:
def simplex_solve_general(A_le=None, b_le=None, c=None, A_ge=None, b_ge=None, A_eq=None, b_eq=None, return_more=False, tol=1e-9):
    """
    Ta funkcja rozwiązuje problem LP w postaci max c^T x przy mieszanych ograniczeniach:
      A_le x ≤ b_le, A_ge x ≥ b_ge, A_eq x = b_eq, x ≥ 0.

    Przyjmuje te same argumenty co preprocess_general_lp:
      A_le, b_le – ograniczenia A_le x ≤ b_le,
      A_ge, b_ge – ograniczenia A_ge x ≥ b_ge,
      A_eq, b_eq – ograniczenia A_eq x = b_eq.
    Poza tym przyjmuje argument typu bool o nazwie return_more.
    Poza tym przyjmuje argument o nazwie tol: tolerancja dla sprawdzania optymalności i unikania dzielenia przez 0 - zmienna typowo do "czysto technicznych celów obliczeń numerycznych".
    Argument tol jest przekazywany do funkcji simplex_core.

    Zwraca:
      x_opt – wektor optymalnych wartości zmiennych (n_orig),
      f_opt – wartość funkcji celu w optimum,
      B_opt – indeksy kolumn bazy w końcowym rozwiązaniu.

    Schemat działania tej funkcji:
     1) Faza I: wywołanie prepare_phase_I_general, uruchomienie simplex_core → uzyskanie x1, f1, B1.
        Jeżeli abs(f1) > 0 → brak dopuszczalnego rozwiązania LP (wywołanie błędu z tym komunikatem)
     2) Budowa modelu fazy II: usunięcie kolumn sztucznych z A_art, skonstruowanie wektora funkcji celu na etapie II
     3) Uzupełnienie bazy za pomocą bazy otrzymanej z fazy I (bez sztucznych zmiennych), ewentualne dodanie slacków, 
        uruchomienie simplex_core fazy II → uzyskanie x2, f2, B2.
     4) Zwrócenie x_opt = pierwsze n_orig współrzędnych x2 oraz (jeśli return_more = True) również f_opt = f2, B_opt = B2.
    """

    # ===================== Faza I =====================
    # Sprawdzamy i przygotowujemy model LP w fazie I (wprowadzamy zmienne sztuczne, ujednolicamy ograniczenia).
    A1, b1, c1, B1, n_orig, m_all = prepare_phase_I_general(A_le, b_le, A_ge, b_ge, A_eq, b_eq)

    # Uruchamiamy rdzeń algorytmu Simplex na modelu fazy I, aby znaleźć pierwsze bazowe rozwiązanie dopuszczalne.
    x1, f1, B1 = simplex_core(A1, b1, c1, B1, return_more=True, tol=tol)

    # Sprawdzamy, czy wartość funkcji celu dla fazy I jest w granicach tolerancji (powinna wynosić zero).
    if abs(f1) > tol:
        # Jeśli wartość jest różna od zera, to problem nie ma rozwiązania dopuszczalnego.
        raise Exception("LP nie ma dopuszczalnych rozwiązań.")

    # ===================== Faza II =====================
    # Sprawdzamy pełną macierz zawierającą zmienne oryginalne, slacki i sztuczne.
    A2 = A1.copy()
    b1_mod = b1.copy()

    # Sprawdzamy liczbę wszystkich zmiennych i ustalamy na której pozycji zaczynają się sztuczne.
    total_vars = A1.shape[1]
    artificial_start = n_orig + m_all

    # Konstruujemy wektor funkcji celu dla fazy II:
    #   • dla zmiennych oryginalnych stosujemy podane c,
    #   • dla slacków i sztucznych ustawiamy wagi na zero.
    c2 = np.vstack([c.reshape((n_orig, 1)), np.zeros((total_vars - n_orig, 1))])

    # Próbujemy utworzyć macierz bazy B_mat oraz jej odwrotność – baza pochodzi z fazy I.
    try:
        B2 = B1.copy()
        B2_set = set(B2)
        B_mat = A2[:, B2]
        # Obliczamy odwrotność macierzy bazowej – posłuży nam do testów pivotu.
        B_mat_inv = np.linalg.inv(B_mat)
    except np.linalg.LinAlgError:
        # Jeśli baza jest osobliwa, w fazie II używamy bazy slacków.
        B2 = list(range(n_orig, n_orig + m_all))
        B2_set = set(B2)
        B_mat = A2[:, B2]
        B_mat_inv = np.linalg.inv(B_mat)

    # Inicjujemy indeks wiersza w bazie
    i = 0
    while i < len(B2):
        base_col = B2[i]

        # Sprawdzamy, czy kolumna bazowa jest zmienną sztuczną (indeks >= artificial_start).
        if base_col >= artificial_start:
            pushed = False  # flaga: czy udało się wypchnąć daną sztuczną zmienną

            # Przeglądamy wszystkie zmienne oryginalne i slacki (indeksy < n_orig + m_all).
            for j in range(n_orig + m_all):
                if j in B2_set:
                    # Pomijamy kolumny, które już są w bazie.
                    continue

                # Obliczamy y_ij = (B_mat_inv @ A2[:, j])[i]
                # To jest współczynnik w i-tym wierszu odpowiadający kolumnie j.
                y_ij = (B_mat_inv @ A2[:, j])[i]

                # Jeśli wartość absolutna y_ij jest większa od tol, to pivotujemy.
                if abs(y_ij) > tol:
                    # Usuwamy bieżącą sztuczną zmienną z zestawu i z listy B2.
                    B2_set.remove(base_col)
                    # Dodajemy kolumnę j do bazy.
                    B2_set.add(j)
                    B2[i] = j
                    pushed = True

                    # Rekalkulujemy macierz bazową i jej odwrotność,
                    # ponieważ baza się zmieniła.
                    B_mat = A2[:, B2]
                    B_mat_inv = np.linalg.inv(B_mat)
                    break  # zakończamy przeszukiwanie w celu wypchnięcia

            # Jeśli nie znaleźliśmy żadnego j z y_ij ≠ 0, to wiersz jest liniowo zależny.
            if not pushed:
                # Usuwamy i-ty wiersz z A2 (ograniczenie nie wnosi nic nowego).
                A2 = np.delete(A2, i, axis=0)
                # Usuwamy odpowiadający element z wektora b1_mod.
                b1_mod = np.delete(b1_mod, i, axis=0)
                # Usuwamy tę sztuczną zmienną z listy B2.
                del B2[i]

                # Ponownie aktualizujemy macierz bazową i jej odwrotność po usunięciu wiersza.
                B_mat = A2[:, B2]
                B_mat_inv = np.linalg.inv(B_mat)

                # Nie zwiększamy i – w bieżącej pozycji w B2 jest nowy wiersz, który trzeba sprawdzić.
                continue

        # Jeśli kolumna nie była sztuczną lub udało się ją wypchnąć, przechodzimy dalej.
        i += 1

    # Po zakończeniu wypychania usuwamy wszystkie kolumny sztuczne z A2 oraz z c2.
    A2 = A2[:, :n_orig + m_all]
    c2 = c2[:n_orig + m_all]

    # ===================== Rozwiązywanie fazy II =====================
    try:
        # Uruchamiamy simplex_core na macierzy oczyszczonej z kolumn sztucznych.
        x2, f2, B2 = simplex_core(A2, b1_mod, c2, B2, return_more=True, tol=tol)
    except Exception as e:
        try:
            # Jeśli nadal baza jest singlularna, przyjmujemy za bazę slacki.
            B2 = list(range(n_orig, n_orig + m_all))
            x2, f2, B2 = simplex_core(A2, b1_mod, c2, B2, return_more=True, tol=tol)
        except Exception as e:
            B2 = None
            x2, f2, B2 = simplex_core(A2, b1_mod, c2, B2, return_more=True, tol=tol)

    # Wyciągamy z x2 tylko pierwsze n_orig współrzędnych – to są wartości zmiennych oryginalnych.
    x_opt = x2[:n_orig].flatten()

    if return_more:
        return x_opt, f2, B2
    else:
        return x_opt

In [37]:
# --- Dwufazowy Simplex (ogólna, ostateczna/końcowa metoda) - WERSJA 2, przed implementacją zabezpieczeń przed szczególnymi przypadkami - POPRAWNA? ---
def simplex_solve_general_faster(A_le = None, b_le = None, c = None, A_ge=None, b_ge=None, A_eq=None, b_eq=None, return_more = False, tol = 1e-9):
    """
    Ta funkcja rozwiązuje problem LP w postaci max c^T x przy mieszanych ograniczeniach:
      A_le x ≤ b_le, A_ge x ≥ b_ge, A_eq x = b_eq, x ≥ 0.

    Przyjmuje te same argumenty co preprocess_general_lp:
      A_le, b_le – ograniczenia A_le x ≤ b_le,
      A_ge, b_ge – ograniczenia A_ge x ≥ b_ge,
      A_eq, b_eq – ograniczenia A_eq x = b_eq.
    Poza tym przyjmuje argument typu bool o nazwie return_more.
    Poza tym przyjmuje argument o nazwie tol: tolerancja dla sprawdzania optymalności i unikania dzielenia przez 0 - zmienna typowo do "czysto technicznych celów obliczeń numerycznych".
    Argument tol jest przekazywany do funkcji simplex_core.

    Zwraca:
      x_opt – wektor optymalnych wartości zmiennych (n_orig),
      f_opt – wartość funkcji celu w optimum,
      B_opt – indeksy kolumn bazy w końcowym rozwiązaniu.

    Schemat działania tej funkcji:
     1) Faza I: wywołanie prepare_phase_I_general, uruchomienie simplex_core → uzyskanie x1, f1, B1.
        Jeżeli abs(f1) > 0 → brak dopuszczalnego rozwiązania LP (wywołanie błędu z tym komunikatem)
     2) Budowa modelu fazy II: usunięcie kolumn sztucznych z A_art, skonstruowanie wektora funkcji celu na etapie II
     3) Uzupełnienie bazy za pomocą bazy otrzymanej z fazy I (bez sztucznych zmiennych), ewentualne dodanie slacków, 
        uruchomienie simplex_core fazy II → uzyskanie x2, f2, B2.
     4) Zwrócenie x_opt = pierwsze n_orig współrzędnych x2 oraz (jeśli return_more = True) również f_opt = f2, B_opt = B2.
    """
    # --- Faza I ---
    A1, b1, c1, B1, n_orig, m_all = prepare_phase_I_general(
        A_le, b_le, A_ge, b_ge, A_eq, b_eq
    )
    x1, f1, B1 = simplex_core(A1, b1, c1, B1, return_more=True, tol=tol)
    if abs(f1) > tol:
        raise Exception("LP nie ma dopuszczalnych rozwiązań.")

    # --- Faza II ---
    # 1) Usuwamy kolumny odpowiadające zmiennym sztucznym: zostawiamy tylko oryginalne zmienne i slacki
    A2 = A1[:, :n_orig + m_all]
    # 2) Wektor współczynników funkcji celu w fazie II: [c (dla x); 0 (dla slack)]
    c2 = np.vstack([c.reshape((n_orig, 1)), np.zeros((m_all, 1))])

    # 3) Pierwotna baza B2 = te kolumny z B1, które są < n_orig+m_all
    B2 = [j for j in B1 if j < n_orig + m_all]

    # 4) Jeżeli brakuje kolumn (len(B2) < m_all), dopisujemy brakujące z przedziału [n_orig .. n_orig+m_all-1]
    if len(B2) < m_all:
        candidates = [j for j in range(n_orig, n_orig + m_all) if j not in B2]
        for j in candidates:
            B2.append(j)
            if len(B2) == m_all:
                break
        # Uwaga: nie nadpisujemy B2 jako pełnego range dla slacków tak "po prostu" – tylko uzupełniamy jakby brakujące wymiary

    # 5) Wywołujemy simplex_core dla fazy II z tą bazą i z tol
    try:
        x2, f2, B2 = simplex_core(A2, b1, c2, B2, return_more=True, tol=tol)
    except:
        B2 = list(range(n_orig, n_orig + m_all))
        x2, f2, B2 = simplex_core(A2, b1, c2, B2, return_more=True, tol=tol)
        
    x_opt = x2[:n_orig].flatten()

    if return_more:
        return x_opt, f2, B2
    else:
        return x_opt

Sprawdźmy działanie całościowej wersji:

In [38]:
simplex_result = simplex_solve_general(A,b,c)
simplex_result #zgadza się z wynikiem z poradnika

array([ 5.6, 25.2])

In [39]:
x_opt, f_opt, B_col_nrs_from_A_opt = tuple(simplex_solve_general(A,b,c, return_more=True))

#zgadza się z wynikami z poradnika
print("x_opt =", x_opt)
print()
print("f_opt =", f_opt)
print()
print("B_col_nrs_from_A_opt =", B_col_nrs_from_A_opt)

x_opt = [ 5.6 25.2]

f_opt = 1484.0

B_col_nrs_from_A_opt = [0, 1, 2]


--- 

Przyjrzyjmy się jeszcze troszkę uważniej fazie I, z formalnego punktu widzenia. W szczególności, uwagę zwrócimy na to, co może podczas rozwiązywania danego problemu LP metodą Simplex, pójść "nie tak".

Warto podkreślić, że w powyższym pseudokodzie macierz $A$ właściwie przy naszych wcześniejszych oznaczeniach jest macierzą $\hat{A}$, podobnie wektor $\bf{x}$ jest właściwie wektorem $\bf{\hat{x}}$.

Niech $[\bf{x}, v]^T$ będzie rozwiązaniem optymalnym problemu wspomnianego w tym pseudokodzie. Mamy trzy możliwości:

1. $-\mathop{\sum}\limits_{i = 1}^{m} v_i < 0$, wówczas istnieje co najmniej jedna sztuczna zmienna o wartości dodatniej. Wówczas wyjściowy problem $LP$ nie ma rozwiązań.

Pokażmy przykład takiego problemu $LP$:

In [40]:
# -1 * x1 - 1 * x2 -> max
# przy ograniczeniach
# x1 + x2 <= 1 --> typu "le"
# x1 + x2 >= 3 --> typu "ge"
# x1, x2 >= 0

A_przykl1_le = np.array([[1,1]]).astype(float)
b_przykl1_le = np.array([[1]]).astype(float)

A_przykl1_ge = np.array([[1,1]]).astype(float)
b_przykl1_ge = np.array([[3]]).astype(float)

A_przykl1, b_przykl1, origin_list_przykl1 = preprocess_general_lp(A_przykl1_le, b_przykl1_le, A_przykl1_ge, b_przykl1_ge, return_origin=True)

In [41]:
A_przykl1

array([[ 1.,  1.],
       [-1., -1.]])

In [42]:
b_przykl1.reshape(-1,1)

array([[ 1.],
       [-3.]])

In [43]:
c_przykl1 = np.array([[-1],
                      [-1]])
c_przykl1

array([[-1],
       [-1]])

Zobaczmy wyniki fazy I:

In [44]:
A_art_przykl1, b_mod_przykl1, c_art_przykl1, B0_przykl1, n_orig_przykl1, m_all_przykl1 = prepare_phase_I_general(A_przykl1,b_przykl1)

In [45]:
A_art_przykl1

array([[ 1.,  1.,  1.,  0.,  1.,  0.],
       [-1., -1.,  0.,  1.,  0., -1.]])

In [46]:
b_mod_przykl1

array([[ 1.],
       [-3.]])

In [47]:
c_art_przykl1

array([[ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [-1.],
       [-1.]])

In [48]:
B0_przykl1

[4, 5]

In [49]:
n_orig_przykl1

2

In [50]:
m_all_przykl1

2

In [51]:
x1_przykl1, f1_przykl1, B1_przykl1 = simplex_core(A_art_przykl1, b_mod_przykl1, c_art_przykl1, B0_przykl1, return_more=True)
if abs(f1_przykl1) > 1e-9:
    print("Brak dopuszczalnego rozwiązania (faza I)")

Brak dopuszczalnego rozwiązania (faza I)


In [52]:
x1_przykl1 #jedna ze zmiennych sztucznych jest dodatnia

array([1., 0., 0., 0., 0., 2.])

In [53]:
f1_przykl1 #faktycznie, funkcja celu przyjmuje ujemną wartość

-2.0

In [54]:
B1_przykl1

[0, 5]

W takiej sytuacji wyjściowy problem $LP$ nie posiada rozwiązań. No i ostateczna metoda w tym przypadku nas o tym informuje:

In [55]:
try:
    simplex_solve_general(A_przykl1, b_przykl1, c_przykl1)
except Exception as e:
    print(e)

LP nie ma dopuszczalnych rozwiązań.


2. $-\mathop{\sum}\limits_{i = 1}^{m} v_i = 0$ i żadna ze zmiennych sztucznych nie ma wartości dodatniej. A więc przechodzimy do fazy II i $\bf{x}$ jest pierwszym bazowym rozwiązaniem dopuszczalnym rozwiązywanego wyjściowo problemu $LP$.

Tak było chociażby w przypadku naszych danych w zadaniu o produkcji lodów.

In [56]:
simplex_solve_general(A, b, c)

array([ 5.6, 25.2])

In [57]:
f1

0.0

In [58]:
B1

[0, 1, 2]

In [59]:
A_hat

array([[ 10.,  20.,   1.,   0.,   0.],
       [ 40.,  30.,   0.,   1.,   0.],
       [100., 200.,   0.,   0.,   1.]])

3. $-\mathop{\sum}\limits_{i = 1}^{m} v_i = 0$ i żadna ze zmiennych sztucznych nie ma wartości dodatniej, ale istnieje co najmniej jedna sztuczna zmienna, która jest zmienną bazową, oznaczmy ją przez $v_{i_\star}$. Oczywiście więc $v_{i_\star} = 0$ oraz $i_\star \in \{ 1, \ldots, m \}$.

Stosujemy wówczas trick, zwany `wypychaniem` zmiennych sztucznych.

Szukamy zmiennej niebazowej $x_k$ (a więc $x_k = 0$), dla której $y_{i\star}^k \not = 0$. 

Zmienna $x_k$ wchodzi do bazy, natomiast zmienna $v_{i_\star} = 0$ wychodzi z bazy. Wówczas $\bar{\Theta}_k = 0$. Przez to, że $\bar{\Theta}_k = 0$, wartość funkcji celu się nie zmienia, ale eliminujemy zmienną sztuczną $v_{i\star}$.

Proces ten powtarzamy, aż wszystkie zmienne sztuczne nie będą zmiennymi bazowymi. 

Otrzymane po takich przekształceniach rozwiązanie $\bf{x}$ będzie bazowym rozwiązaniem dopuszczalnym, ale zdegenerowanym (tzn. zawierającym zera na co najmniej jednej współrzędnej - z prostej przyczyny, bo włączamy do rozwiązania zmiennie niebazowe - ale już przynajmniej nie sztuczne).

In [60]:
# x1 -> max
# przy ograniczeniach
# x1 = 1
# x1 + x2 = 1
# x1, x2 >= 0

A_przykl2_eq = np.array([[1,0],
                         [1,1]]).astype(float)
b_przykl2_eq = np.array([[1],
                         [1]]).astype(float)

A_przykl2, b_przykl2, origin_list_przykl2 = preprocess_general_lp(A_eq=A_przykl2_eq, b_eq=b_przykl2_eq, return_origin=True)

In [61]:
A_przykl2

array([[ 1.,  0.],
       [-1., -0.],
       [ 1.,  1.],
       [-1., -1.]])

In [62]:
np.linalg.matrix_rank(A_przykl2) #macierz A jest pełnego rzędu

np.int64(2)

In [63]:
b_przykl2.reshape(-1,1)

array([[ 1.],
       [-1.],
       [ 1.],
       [-1.]])

In [64]:
origin_list_przykl2

['eq', 'eq', 'eq', 'eq']

In [65]:
c_przykl2 = np.array([[1],
                      [0]])

In [66]:
A_art_przykl2, b_mod_przykl2, c_art_przykl2, B0_przykl2, n_orig_przykl2, m_all_przykl2 = prepare_phase_I_general(A_przykl2,b_przykl2)

In [67]:
A_art_przykl2

array([[ 1.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [-1., -0.,  0.,  1.,  0.,  0.,  0., -1.,  0.,  0.],
       [ 1.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.],
       [-1., -1.,  0.,  0.,  0.,  1.,  0.,  0.,  0., -1.]])

In [68]:
np.linalg.matrix_rank(A_art_przykl2)

np.int64(4)

In [69]:
b_mod_przykl2

array([[ 1.],
       [-1.],
       [ 1.],
       [-1.]])

In [70]:
c_art_przykl2

array([[ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [-1.],
       [-1.],
       [-1.],
       [-1.]])

In [71]:
B0_przykl2

[6, 7, 8, 9]

In [72]:
n_orig_przykl2

2

In [73]:
m_all_przykl2

4

In [74]:
x1_przykl2, f1_przykl2, B1_przykl2 = simplex_core(A_art_przykl2, b_mod_przykl2, c_art_przykl2, B0_przykl2, return_more=True)
if abs(f1_przykl2) > 1e-9:
    raise Exception("Brak dopuszczalnego rozwiązania (faza I)")

In [75]:
x1_przykl2

array([ 1.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0., -0.])

In [76]:
f1_przykl2

0.0

In [77]:
B1_przykl2

[0, 1, 7, 9]

In [78]:
simplex_solve_general(A_eq=A_przykl2_eq, b_eq=b_przykl2_eq, c = c_przykl2)

array([1., 0.])

Powyższy proces "wypychania" zmiennnej sztucznej $v_{i\star}$ nie powiedzie się tylko, jeśli $y_{i_\star}^k = 0$ dla wszystkich kolumn odpowiadających zmiennym niesztucznym.

Ale to oznaczałoby, że istnieją przekształcenia elementarne zerujące wiersz $i_\star$ w oryginalnej macierzy $\hat{A}$, a więc $\mathop{\text{rank}} \hat{A} < m$. 

W takim przypadku usuwamy $i_\star$-ty wiersz z macierzy $\hat{A}$ (a więc usuwamy $i_\star$-te ograniczenie, bo jest ono liniowo zależne od pozostałych, więc nic nowego nie wnosi). 

In [79]:
# x1 -> max
# przy ograniczeniach
# x1 + x2 = 2
# 2x1 + 2x2 = 4
# x1, x2 >= 0

A_przykl3_eq = np.array([[1,1],
                         [2,2]]).astype(float)
b_przykl3_eq = np.array([[2],
                         [4]]).astype(float)

A_przykl3, b_przykl3, origin_list_przykl3 = preprocess_general_lp(A_eq=A_przykl3_eq, b_eq=b_przykl3_eq, return_origin=True)

In [80]:
A_przykl3

array([[ 1.,  1.],
       [-1., -1.],
       [ 2.,  2.],
       [-2., -2.]])

In [81]:
np.linalg.matrix_rank(A_przykl3) #tutaj nie mamy pełnego rzędu

np.int64(1)

In [82]:
b_przykl3

array([ 2., -2.,  4., -4.])

In [83]:
origin_list_przykl3

['eq', 'eq', 'eq', 'eq']

In [84]:
c_przykl3 = np.array([[1],
                      [0]])

In [85]:
A_art_przykl3, b_mod_przykl3, c_art_przykl3, B0_przykl3, n_orig_przykl3, m_all_przykl3 = prepare_phase_I_general(A_przykl3,b_przykl3)

In [86]:
A_art_przykl3

array([[ 1.,  1.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [-1., -1.,  0.,  1.,  0.,  0.,  0., -1.,  0.,  0.],
       [ 2.,  2.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.],
       [-2., -2.,  0.,  0.,  0.,  1.,  0.,  0.,  0., -1.]])

In [87]:
np.linalg.matrix_rank(A_art_przykl3)

np.int64(4)

In [88]:
b_mod_przykl3

array([[ 2.],
       [-2.],
       [ 4.],
       [-4.]])

In [89]:
c_art_przykl3

array([[ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [-1.],
       [-1.],
       [-1.],
       [-1.]])

In [90]:
B0_przykl3

[6, 7, 8, 9]

In [91]:
n_orig_przykl3

2

In [92]:
m_all_przykl3

4

In [93]:
x1_przykl3, f1_przykl3, B1_przykl3 = simplex_core(A_art_przykl3, b_mod_przykl3, c_art_przykl3, B0_przykl3, return_more=True)
if abs(f1_przykl3) > 1e-9:
    raise Exception("Brak dopuszczalnego rozwiązania (faza I)")

In [94]:
x1_przykl3

array([ 2.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0., -0.])

In [95]:
f1_przykl3

0.0

In [96]:
B1_przykl3

[0, 4, 7, 9]

In [97]:
simplex_solve_general(A_eq=A_przykl3_eq, b_eq=b_przykl3_eq, c = c_przykl3)

array([2., 0.])

Przytaczamy teraz na sam koniec kilka uwag na temat możliwych przyczyn degeneracji rozwiązania z wykładu [1]:

![image.png](projekt_alg_Simplex_zdj/degeneracja_tekst.png)

Żeby nie wpaść w cykl, stosuje się właśnie powyżej wspomniane:

`Twierdzenie 4 (Algorytm Simplex zatrzymuje się)` Załóżmy, że $N_k$ będzie kolumną wchodzącą do bazy wybraną następująco:

$$ k = \mathop{\text{min}} \{ j \colon c_j - z_j > 0, m + 1 \le j \le m + n\}$$

oraz w przypadku pojawienia się niejednoznaczności przy zastosowaniu kryterium wyjścia, przyjmiemy jako indeks zmiennej wychodzącej z bazy indeks $i_{\star}$ będący indeksem o najmniejszej wartości spośród indeksów wyznaczających minimalną wartość ilorazu $\frac{x_{i_\star}^B}{y_{i_\star}^k}$. Wówczas algorytm sympleks skończy działanie w skończonej liczbie kroków.

Opisany powyżej sposób wyboru zmiennych (o najmniejszych indeksach) znany jest jako `reguła Bland’a`.

Na koniec porównanie wyników na jeszcze jednym przykładzie:

In [98]:
# --- PRZYKŁAD PROBLEMU LP z ograniczeniami mieszanych typów <= i >= i == ---
# max x1 + 2 x2
#   x1 + x2 <= 4
#   x1 - x2 >= 2       -->  -x1 + x2 <= -2
#   x1 - 3 x2 = 1      -->  równanie

# 1) A_le x ≤ b_le
A_le = np.array([[1, 1]])    # x₁ + x₂ ≤ 4
b_le = np.array([4])
# 2) A_ge x ≥ b_ge
A_ge = np.array([[1, -1]])   # x₁ − x₂ ≥ 2
b_ge = np.array([2])
# 3) A_eq x = b_eq
A_eq = np.array([[1, -3]])   # x₁ − 3x₂ = 1
b_eq = np.array([1])
# Wektor współczynników funkcji celu: max (1*x₁ + 2*x₂)
c = np.array([1, 2])

# Rozwiązanie własnym simplexem dwuetapowym
x_sol, f_sol, B_sol = simplex_solve_general(
    A_le, b_le, c,
    A_ge=A_ge, b_ge=b_ge,
    A_eq=A_eq, b_eq=b_eq, return_more=True
)
print('nasz solver - rozwiązanie:          {},               wartość funkcji celu: {}'.format(x_sol, f_sol))

# Porównanie z SciPy (Highs)
A_ub, b_ub = preprocess_general_lp(A_le, b_le, A_ge, b_ge, A_eq, b_eq)
res = linprog(c=-c, A_ub=A_ub, b_ub=b_ub,
                bounds=[(0, None)] * c.shape[0], method='highs')
print('scipy solver linprog - rozwiązanie: {},               wartość funkcji celu: {}'.format(res.x, -res.fun))
# Uwaga: SciPy dla maksymalizacji przyjmuje -c i zwraca -fun jako wartość celu.

nasz solver - rozwiązanie:          [3.25 0.75],               wartość funkcji celu: 4.75
scipy solver linprog - rozwiązanie: [3.25 0.75],               wartość funkcji celu: 4.75


---

Inne testy na różnych przypadkach (wszystkie te testy algorytm przechodzi pomyślnie):

In [99]:
import numpy as np
# maximize x1 + x2
#  x1 + 2x2 ≤ 4
#  4x1 + 2x2 ≥ 6
#  x1 - x2  = 1
#  x1, x2 ≥ 0
A_le = np.array([[1, 2]])
b_le = np.array([4])
A_ge = np.array([[4, 2]])
b_ge = np.array([6])
A_eq = np.array([[1, -1]])
b_eq = np.array([1])
c = np.array([1, 1])
x_opt, f_opt, B_opt = simplex_solve_general(A_le, b_le, c, A_ge, b_ge, A_eq, b_eq, return_more=True)
print("x_opt =", x_opt)
print("f_opt =", f_opt)

x_opt = [2. 1.]
f_opt = 3.0


In [100]:
import numpy as np

# Ograniczenia ≤:
A_le = np.array([[1, 1]])
b_le = np.array([1])

# Ograniczenia ≥:
A_ge = np.array([[1, 1]])
b_ge = np.array([3])

# Brak równań:
A_eq = None
b_eq = None

# Funkcja celu:
c = np.array([1, 1])

try:
    x_opt = simplex_solve_general(A_le, b_le, c, A_ge, b_ge, A_eq, b_eq)
    print("Nieoczekiwanie znaleziono rozwiązanie:", x_opt)
except Exception as e:
    print("Oczekiwany wyjątek:", e)

Oczekiwany wyjątek: LP nie ma dopuszczalnych rozwiązań.


In [101]:
import numpy as np

# Ograniczenia ≤:
A_le = np.array([[1, -1]])
b_le = np.array([1])

# Bez ograniczeń ≥ i =:
A_ge = None
b_ge = None
A_eq = None
b_eq = None

# Funkcja celu: max x1 (czyli c = [1, 0])
c = np.array([1, 0])

try:
    x_opt, f_opt, B_opt = simplex_solve_general(A_le, b_le, c, A_ge, b_ge, A_eq, b_eq, return_more=True)
    print("Znalezione rozwiązanie:", x_opt, "f =", f_opt)
    print("⟵ To oznacza, że algorytm nie wykrył nieograniczoności.")
except Exception as e:
    print("Oczekiwany wyjątek (lub informacja o nieograniczoności):", e)

Oczekiwany wyjątek (lub informacja o nieograniczoności): LP nieograniczone


In [102]:
import numpy as np

# Ograniczenia ≤:
A_le = np.array([[1, 0],
                 [2, 0]])
b_le = np.array([5, 10])

# Ograniczenia ≥:
A_ge = np.array([[1, 0]])
b_ge = np.array([1])

# Brak równań:
A_eq = None
b_eq = None

# Funkcja celu: max x1 (czyli c = [1, 0])
c = np.array([1, 0])

x_opt, f_opt, B_opt = simplex_solve_general(A_le, b_le, c, A_ge, b_ge, A_eq, b_eq, return_more=True)
print("Rozwiązanie:", x_opt)    # spodziewane x1=5, x2=0
print("Wartość funkcji celu:", f_opt)

Rozwiązanie: [5. 0.]
Wartość funkcji celu: 5.0


In [103]:
import numpy as np

A_le = np.array([[1, 1],
                 [1, 0]])
b_le = np.array([2, 1])

A_ge = None
b_ge = None
A_eq = None
b_eq = None

c = np.array([1, 1])

x_opt, f_opt, B_opt = simplex_solve_general(A_le, b_le, c, A_ge, b_ge, A_eq, b_eq, return_more=True)
print("Rozwiązanie:", x_opt)    # powinno dać x1=1, x2=1
print("Wartość celu:", f_opt, "Baza:", B_opt)

Rozwiązanie: [1. 1.]
Wartość celu: 2.0 Baza: [0, 1]


In [104]:
import numpy as np

A_le = np.array([[1, 0, 1, 0, 0],
                 [0, 1, 0, 1, 0],
                 [1, 1, 0, 0, 1]])
b_le = np.array([2, 3, 4])

A_ge = None
b_ge = None
A_eq = None
b_eq = None

c = np.array([15, 10, 0, 0, 0])

x_opt, f_opt, B_opt = simplex_solve_general(A_le, b_le, c, A_ge, b_ge, A_eq, b_eq, return_more=True)
print("Rozwiązanie:", x_opt)    # powinno dać (x1,x2, x3, x4, x5) = (2, 2, 0, 1, 0), f_opt = 50
print("Wartość celu:", f_opt, "Baza:", B_opt)

Rozwiązanie: [2. 2. 0. 1. 0.]
Wartość celu: 50.0 Baza: [0, 1, 3]
